In [9]:
import requests
import datetime
import re # regular expression
import os
# ---------------
# 偽裝成瀏覽器
# # ---------------
# headers = {
#     'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
#     '1 Firefox/4.0.1',
#     'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#     'Accept-Language':'en-us,en;q=0.5',
#     'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

# ---------------
# 開啟有驗證碼的網頁
# ---------------
# rs = requests.session()

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------

    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group()[24: ]
#     print('get_verify_code OK')
    return viewstate, eventvalidation

# ---------------
# 爬取資料
# ---------------

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__VIEWSTATEGENERATOR': 'AA1F01CB',
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'V39RG',#'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "http://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

# res = get_data(rs, '1101', viewstate, eventvalidation)
# res.encoding='big5'
# open('1102.csv', 'w').write(res.text)
# print(res.text)
# 3. 整理資料
import pandas as pd
from io import StringIO
import time
def parse_data(text):
    lines = text.split('\n')
    lines = [l for l in lines if len(l.split(',')) == 11]
#     print('transform to dataframe')
    df = pd.read_csv(StringIO('\n'.join(lines)))

    first_df = df[df.columns[:5]]
    second_df = df[df.columns[6:]]
    second_df.columns = second_df.columns.str.replace('.1', '')
    final_df = first_df.append(second_df).set_index('序號').sort_index().dropna()
#     print('transform OK')
    return final_df

In [ ]:
import time
import requests
from io import StringIO
import pandas as pd
import datetime

def download_stock(stock_id, path):
    rs = requests.Session()
    viewstate, eventvalidation = get_verify_code(rs)
    time.sleep(1)
    res = get_data(rs, stock_id, viewstate, eventvalidation)
    i=0
    while (res.text is None or res.text == '') and i<3 :
        print(stock_id,'is empty, crawl again',i+1)
        rs = requests.Session()
        viewstate, eventvalidation = get_verify_code(rs)
        time.sleep(2)
        res = get_data(rs, stock_id, viewstate, eventvalidation)
        i+=1
    res.encoding='big5'
    open(path, 'w', encoding='utf-8').writelines(res.text)

def stock_list():
    res = requests.get('http://dts.twse.com.tw/opendata/t187ap03_L.csv')
    res.encoding='big5'
    df = pd.read_csv(StringIO(res.text), index_col=['公司代號'])
    return df

# build main dir
main_dir = os.path.join('data', 'broker_history')
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
    
# # build date dir
now = datetime.datetime.now()
date_dir = os.path.join(main_dir,now.strftime('%Y%m%d'))
if not os.path.isdir(date_dir):
    os.mkdir(date_dir)

startT = time.time()    
slist = stock_list()
lens = len(slist)
slist.index
for s in slist.index: 
    stock_path = os.path.join(date_dir, str(s) + '.csv')
    print(stock_path)
    if not os.path.isfile(stock_path):
        count = slist.index.get_loc(s)+1
        print('downloading...',count,'/',lens)
        download_stock(str(s), stock_path)
        time.sleep(2)
        
print('download complete/n transform to dataframe')
import shutil
bak_dir = os.path.join('data', 'broker_history_bak',now.strftime('%Y%m%d'))
shutil.copytree(date_dir,bak_dir)

for t in slist.index:
    count = slist.index.get_loc(t)+1
    print('transform...',t,count,'/',lens)
    stock_path = os.path.join(date_dir, str(t) + '.csv')
    with open(stock_path , 'r', encoding='UTF-8') as file:
        if os.stat(stock_path).st_size != 0 :
            context = file.read() 
        else :
            print(t," is empty")
            continue
    df2 = parse_data(context)
    df2.to_csv(stock_path, encoding='utf_8_sig')
# time.sleep(1)
endT = time.time()  
print(round(endT-startT,2),"sec")  

data\broker_history\20180711\1101.csv
data\broker_history\20180711\1102.csv
data\broker_history\20180711\1103.csv
data\broker_history\20180711\1104.csv
data\broker_history\20180711\1108.csv
data\broker_history\20180711\1109.csv
data\broker_history\20180711\1110.csv
data\broker_history\20180711\1201.csv
data\broker_history\20180711\1203.csv
data\broker_history\20180711\1210.csv
data\broker_history\20180711\1213.csv
data\broker_history\20180711\1215.csv
data\broker_history\20180711\1216.csv
data\broker_history\20180711\1217.csv
data\broker_history\20180711\1218.csv
data\broker_history\20180711\1219.csv
data\broker_history\20180711\1220.csv
data\broker_history\20180711\1225.csv
data\broker_history\20180711\1227.csv
data\broker_history\20180711\1229.csv
data\broker_history\20180711\1231.csv
data\broker_history\20180711\1232.csv
data\broker_history\20180711\1233.csv
data\broker_history\20180711\1234.csv
data\broker_history\20180711\1235.csv
data\broker_history\20180711\1236.csv
data\broker_

data\broker_history\20180711\1617.csv
downloading... 151 / 934
data\broker_history\20180711\1618.csv
downloading... 152 / 934
data\broker_history\20180711\1626.csv
downloading... 153 / 934
data\broker_history\20180711\1701.csv
downloading... 154 / 934
data\broker_history\20180711\1702.csv
downloading... 155 / 934
data\broker_history\20180711\1704.csv
downloading... 156 / 934
data\broker_history\20180711\1707.csv
downloading... 157 / 934
data\broker_history\20180711\1708.csv
downloading... 158 / 934
data\broker_history\20180711\1709.csv
downloading... 159 / 934
data\broker_history\20180711\1710.csv
downloading... 160 / 934
data\broker_history\20180711\1711.csv
downloading... 161 / 934
data\broker_history\20180711\1712.csv
downloading... 162 / 934
data\broker_history\20180711\1713.csv
downloading... 163 / 934
data\broker_history\20180711\1714.csv
downloading... 164 / 934
data\broker_history\20180711\1717.csv
downloading... 165 / 934
data\broker_history\20180711\1718.csv
downloading... 16

data\broker_history\20180711\2344.csv
downloading... 278 / 934
data\broker_history\20180711\2345.csv
downloading... 279 / 934
data\broker_history\20180711\2347.csv
downloading... 280 / 934
data\broker_history\20180711\2348.csv
downloading... 281 / 934
data\broker_history\20180711\2349.csv
downloading... 282 / 934
data\broker_history\20180711\2351.csv
downloading... 283 / 934
data\broker_history\20180711\2352.csv
downloading... 284 / 934
data\broker_history\20180711\2353.csv
downloading... 285 / 934
data\broker_history\20180711\2354.csv
downloading... 286 / 934
data\broker_history\20180711\2355.csv
downloading... 287 / 934
data\broker_history\20180711\2356.csv
downloading... 288 / 934
data\broker_history\20180711\2357.csv
downloading... 289 / 934
data\broker_history\20180711\2358.csv
downloading... 290 / 934
data\broker_history\20180711\2359.csv
downloading... 291 / 934
data\broker_history\20180711\2360.csv
downloading... 292 / 934
data\broker_history\20180711\2362.csv
downloading... 29

data\broker_history\20180711\2527.csv
downloading... 407 / 934
data\broker_history\20180711\2528.csv
downloading... 408 / 934
data\broker_history\20180711\2530.csv
downloading... 409 / 934
data\broker_history\20180711\2534.csv
downloading... 410 / 934
data\broker_history\20180711\2535.csv
downloading... 411 / 934
data\broker_history\20180711\2536.csv
downloading... 412 / 934
data\broker_history\20180711\2537.csv
downloading... 413 / 934
data\broker_history\20180711\2538.csv
downloading... 414 / 934
data\broker_history\20180711\2539.csv
downloading... 415 / 934
data\broker_history\20180711\2540.csv
downloading... 416 / 934
data\broker_history\20180711\2542.csv
downloading... 417 / 934
data\broker_history\20180711\2543.csv
downloading... 418 / 934
data\broker_history\20180711\2545.csv
downloading... 419 / 934
data\broker_history\20180711\2546.csv
downloading... 420 / 934
data\broker_history\20180711\2547.csv
downloading... 421 / 934
data\broker_history\20180711\2548.csv
downloading... 42

data\broker_history\20180711\3031.csv
downloading... 532 / 934
data\broker_history\20180711\3032.csv
downloading... 533 / 934
data\broker_history\20180711\3033.csv
downloading... 534 / 934
data\broker_history\20180711\3034.csv
downloading... 535 / 934
data\broker_history\20180711\3035.csv
downloading... 536 / 934
data\broker_history\20180711\3036.csv
downloading... 537 / 934
data\broker_history\20180711\3037.csv
downloading... 538 / 934
data\broker_history\20180711\3038.csv
downloading... 539 / 934
3038 is empty, crawl again 1
data\broker_history\20180711\3040.csv
downloading... 540 / 934
data\broker_history\20180711\3041.csv
downloading... 541 / 934
data\broker_history\20180711\3042.csv
downloading... 542 / 934
data\broker_history\20180711\3043.csv
downloading... 543 / 934
data\broker_history\20180711\3044.csv
downloading... 544 / 934
data\broker_history\20180711\3045.csv
downloading... 545 / 934
3045 is empty, crawl again 1
data\broker_history\20180711\3046.csv
downloading... 546 / 9

# 券商總買賣

In [71]:
df = pd.read_csv('1102.csv')
buy = df['買進股數'].astype(int).groupby(df['券商']).sum()
sell= df['賣出股數'].astype(int).groupby(df['券商']).sum()
total = buy - sell
total

券商
1020合　　庫        2360
1021合庫台中        2000
1022合庫台南        1000
1024合庫嘉義       -2000
1028合庫彰化       -1000
1029合庫鳳山        5000
102A合庫新竹        1000
102C合庫自強        3000
102G合庫三重       -7000
1031土銀台中        1000
1033土銀高雄        9000
1034土銀嘉義        2000
1037土銀花蓮        5000
103F土銀南港        2000
1040臺    銀    246200
1041臺銀鳳山      -40801
1042臺銀臺南           0
1043臺銀民權      -25000
1045臺銀新竹        2000
104A臺銀臺中        9000
104C臺銀高雄        1000
1110企    銀     19000
1114企銀嘉義        5795
1117企銀竹北        2000
1119企銀岡山        2000
111E企銀桃園       27000
1160日    盛    568300
1161日盛忠孝           0
1162日盛台南           0
1163日盛台中        1000
               ...  
9A8F永豐敦南         -97
9A91永豐松山        3000
9A93永豐大稻       30000
9A96永豐博愛        6000
9A9A永豐羅東      -20000
9A9C永豐員林       -4000
9A9D永豐忠孝       -1000
9A9E永豐板橋       -1000
9A9F永豐安和        6000
9A9J永豐板新        2274
9A9K永豐三重        1000
9A9L永豐台中           0
9A9Q永豐豐原       -1000
9A9R永豐信義        1000
9A9S永豐南京        1000
9A9V永豐三民        5000
9A9X永豐竹科  

# 主力買賣超

In [13]:
# 買方top 15
total.nlargest(15)

券商
1440美林        806000
9200凱基        465000
1380台灣匯立      454000
1480美商高盛      444000
8960上海匯豐      422000
8440摩根大通      393000
9307華南大安      320000
1470台灣摩根      318000
1040臺    銀    283000
8560新    光    270000
9813元大八德      160000
5850統    一    150000
9695富邦民雄      140000
981N元大金門      112000
8840玉    山    100000
dtype: int64

In [162]:
# 賣方 top 15
total.nsmallest(15)

券商
8440摩根大通     -3093000
1590花旗環球     -1979000
8880國泰綜合     -1769000
9200凱基       -1497000
9800元大        -958000
9300華南永昌      -889000
9131群益民權      -355000
8580聯    邦    -340000
8840玉    山    -315000
779c國票敦北      -254000
6450永    全    -250000
1233彰銀台中      -240000
7001兆豐嘉義      -186000
5600永    興    -123000
9211凱基台南      -104000
dtype: int64

In [166]:
# 主力買賣超
total.nlargest(15).sum() + total.nsmallest(15).sum()

-559605

# 買賣家數差

In [164]:
# 買賣家數差
(total > 0).sum() - (total < 0).sum()

45